In [2]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# 기본 Retriever

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter




# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
# 문서 로드
loader = TextLoader("./data/text.txt", encoding="utf-8")
# 문서 분할
split_docs = loader.load_and_split(text_splitter)

# 임베딩 모델 설정
model_name = "intfloat/multilingual-e5-large-instruct"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

# 문서로부터 FAISS
db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)

    # 5. 리트리버 생성
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 30, "lambda_mult": 0.25, "fetch_k": 50}
)

# 질의 설정
query = "삼성전자 하반기 실적 전망은?"

# 질의 수행 및 결과 문서 반환
docs = retriever.invoke(query)

# 결과 문서 출력
pretty_print_docs(docs)

Document 1:

특히 파운드리 고객사를 확보한 것이 단기 실적에는 영향을 미치지 않지만, 중장기 경쟁력 회복을 위한 기반이 될 것이라고 평가했다. 류 연구원은 "여러 기대감이 모여 밸류에이션이 정상화되는 국면"이라고 분석했다. 
이어 3분기 영업이익은 9조원으로 1년 전보다는 1.9% 감소하지만 직전 분기 대비 91.9% 늘어날 것으로 낙관했다.
미래에셋증권은 15일 보고서에서 삼성전자의 올해와 내년 영업이익 전망치를 기존대비 각각 2.9%, 5.3%씩 올렸다. 동시에 목표 주가를 기존 8만8000원에서 9만6000원으로 9% 높였다.
미래에셋증권은 주가 상향 이유로 내년 메모리 반도체 공급 부족을 꼽았다. 반도체 업체들은 제한된 D램 생산라인을 갖고 있는데, 업황 불확실성 탓에 설비를 크게 늘리지 못했다. 새로 증설한 시설도 대부분 차세대 반도체인 고대역폭 메모리(HBM) 생산에 집중돼 있어 범용 D램의 생산량이 줄면서 가격이 오르고 있다.
----------------------------------------------------------------------------------------------------
Document 2:

일탈 조항
국제회계기준 IFRS17의 원칙적인 적용이 어려운 특수한 상황에서 예외적인 회계처리를 허용하는 규정. 일탈 조항의 적용을 받지 못하게 되면 삼성생명 측은 향후 삼성전자 주식을 어떻게 매각해서 계약자에게 돌려줘야 할지 계획을 세워야 하는 상황에 몰리게 된다.
[박창영 기자]"절차 정당성·구속력에 문제"
국제회계기준 부합도 따져야
삼성생명의 회계처리 방식과 관련해 한국회계기준원이 적용의견서 발표에 속도를 내는 가운데 회계학 전문가들 사이에서는 절차적 정당성이나 국제호계기준과 합치 여부, 기업과 시장의 혼란 가능성 등에 대한 우려도 나온다.
-----------------------------------------------------------------------------------------------

# Reranker 사용

In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=6)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

query = '삼성전자 하반기 실적 전망은?'
# 압축된 문서 검색
compressed_docs = compression_retriever.invoke(query)

# 문서 출력
pretty_print_docs(compressed_docs)

Document 1:

특히 파운드리 고객사를 확보한 것이 단기 실적에는 영향을 미치지 않지만, 중장기 경쟁력 회복을 위한 기반이 될 것이라고 평가했다. 류 연구원은 "여러 기대감이 모여 밸류에이션이 정상화되는 국면"이라고 분석했다. 
이어 3분기 영업이익은 9조원으로 1년 전보다는 1.9% 감소하지만 직전 분기 대비 91.9% 늘어날 것으로 낙관했다.
미래에셋증권은 15일 보고서에서 삼성전자의 올해와 내년 영업이익 전망치를 기존대비 각각 2.9%, 5.3%씩 올렸다. 동시에 목표 주가를 기존 8만8000원에서 9만6000원으로 9% 높였다.
미래에셋증권은 주가 상향 이유로 내년 메모리 반도체 공급 부족을 꼽았다. 반도체 업체들은 제한된 D램 생산라인을 갖고 있는데, 업황 불확실성 탓에 설비를 크게 늘리지 못했다. 새로 증설한 시설도 대부분 차세대 반도체인 고대역폭 메모리(HBM) 생산에 집중돼 있어 범용 D램의 생산량이 줄면서 가격이 오르고 있다.
----------------------------------------------------------------------------------------------------
Document 2:

증권업계는 올해 '9만 전자' 달성도 가능하다고 전망한다. 16일 금융투자업계에 따르면, 키움증권(9만원), KB증권(9만원), BNK투자증권(9만1000원), NH투자증권(9만4000원), 한국투자증권(9만5000원), 미래에셋증권(9만6000원) 등이 삼성전자 목표가를 9만원 이상으로 제시했다. 
특히 NH투자증권은 16일 삼성전자 목표 주가를 기존보다 11.9% 상향했다. 류영호 NH투자증권 연구원은 16일 관련 보고서에서 "최근 파운드리 부문에서 테슬라와 애플 등 의미 있는 고객사가 확보된 점, 1c(10나노급 6세대) 수율 개선과 함께 하반기 엔비디아 고대역폭 메모리(HBM) 공급이 가능할 것이란 기대감 등에 힘입어 주가가 긍정적 흐름을 보이고 있다"며 이같이 밝혔다.
--------------------

In [6]:
compressed_docs

[Document(id='3d538354-ad2c-420b-93e5-d924a6088e5c', metadata={'source': './data/text.txt'}, page_content='특히 파운드리 고객사를 확보한 것이 단기 실적에는 영향을 미치지 않지만, 중장기 경쟁력 회복을 위한 기반이 될 것이라고 평가했다. 류 연구원은 "여러 기대감이 모여 밸류에이션이 정상화되는 국면"이라고 분석했다. \n이어 3분기 영업이익은 9조원으로 1년 전보다는 1.9% 감소하지만 직전 분기 대비 91.9% 늘어날 것으로 낙관했다.\n미래에셋증권은 15일 보고서에서 삼성전자의 올해와 내년 영업이익 전망치를 기존대비 각각 2.9%, 5.3%씩 올렸다. 동시에 목표 주가를 기존 8만8000원에서 9만6000원으로 9% 높였다.\n미래에셋증권은 주가 상향 이유로 내년 메모리 반도체 공급 부족을 꼽았다. 반도체 업체들은 제한된 D램 생산라인을 갖고 있는데, 업황 불확실성 탓에 설비를 크게 늘리지 못했다. 새로 증설한 시설도 대부분 차세대 반도체인 고대역폭 메모리(HBM) 생산에 집중돼 있어 범용 D램의 생산량이 줄면서 가격이 오르고 있다.'),
 Document(id='2eb2f80d-33c1-40b5-8ab3-0df3d15eb08f', metadata={'source': './data/text.txt'}, page_content='시장이 기대하는 삼성의 마지막 ‘한방’은 밸류에이션이다. 삼성전자는 여전히 밸류에이션에서 글로벌 경쟁사 대비 큰 할인 상태다. SK하이닉스가 주가순자산비율(PBR) 1.5배 이상에서 거래되는 반면, 삼성전자는 1배 수준에 머물러 있다. SK하이닉스도 저평가되어 있지만 삼전은 하이닉스와의 밸류에이션 갭이 50% 이상 벌어진 상황이다.\n김동원 KB증권 리서치본부장은  “밸류에이션 측면에서 삼성은 하이닉스 대비 여전히 디스카운트 상태라 HBM4와 파운드리 등 여러 조건이 맞아떨어진다면 추가 상승 여력을 기대해도 좋다”고 말했다.이재용 주식가치, 상속 이후